In [ ]:
# export
from nbdev.imports import *
from nbdev.export import *

In [ ]:
# default_exp merge

# Fix merge conflicts

> Fix merge conflicts in jupyter notebooks

When working with jupyter notebooks (which are json files behind the scenes) and GitHub, it is very common that a merge conflict (that will add new lines in the notebook source file) will break some notebooks you are working on. This module defines the function `fix_conflicts` to fix those notebooks for you, and attempt to automatically merge standard conflicts. The remaining ones will be delimited by markdown cells like this:

<img alt="Fixed notebook" width="700" caption="A notebook fixed after a merged conflict. The file couldn't be opened before the command was run, but after it the conflict is higlighted by markdown cells." src="images/merge.PNG" />

In [ ]:
#export
def extract_cells(raw_txt):
    "Manually extract cells in potential broken json `raw_txt`"
    lines = raw_txt.split('\n')
    cells = []
    i = 0
    while not lines[i].startswith(' "cells"'): i+=1
    i += 1
    start = '\n'.join(lines[:i])
    while lines[i] != ' ],':
        while lines[i] != '  {': i+=1
        j = i
        while not lines[j].startswith('  }'): j+=1
        c = '\n'.join(lines[i:j+1])
        if not c.endswith(','): c = c + ','
        cells.append(c)
        i = j+1
    end = '\n'.join(lines[i:])
    return start,cells,end

In [ ]:
#export
conflicts = '<<<<<<< ======= >>>>>>>'.split()

In [ ]:
#export
def get_md_cell(txt):
    "A markdown cell with `txt`"
    return '''  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "''' + txt + '''"
   ]
  },'''

In [ ]:
tst = '''  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "A bit of markdown"
   ]
  },'''
assert get_md_cell("A bit of markdown") == tst

In [ ]:
#export
def _split_cell(cell, cf, names):
    res1,res2 = [],[]
    for line in cell.split('\n'):
        if line.startswith(conflicts[cf]):
            if names[cf//2] is None: names[cf//2] = line[8:]
            cf = (cf+1)%3
            continue
        if cf<2:    res1.append(line)
        if cf%2==0: res2.append(line)
    return '\n'.join(res1),'\n'.join(res2),cf,names

In [ ]:
#hide
tst = '\n'.join(['a', f'{conflicts[0]} HEAD', 'b', conflicts[1], 'c', f'{conflicts[2]} lala', 'd'])
v1,v2,cf,names = _split_cell(tst, 0, [None,None])
assert v1 == 'a\nb\nd'
assert v2 == 'a\nc\nd'
assert cf == 0
assert names == ['HEAD', 'lala']

In [ ]:
#hide
tst = '\n'.join(['a', f'{conflicts[0]} HEAD', 'b', conflicts[1], 'c', f'{conflicts[2]} lala', 'd', f'{conflicts[0]} HEAD', 'e'])
v1,v2,cf,names = _split_cell(tst, 0, [None,None])
assert v1 == 'a\nb\nd\ne'
assert v2 == 'a\nc\nd'
assert cf == 1
assert names == ['HEAD', 'lala']

In [ ]:
#hide
tst = '\n'.join(['a', f'{conflicts[0]} HEAD', 'b', conflicts[1], 'c', f'{conflicts[2]} lala', 'd', f'{conflicts[0]} HEAD', 'e', conflicts[1]])
v1,v2,cf,names = _split_cell(tst, 0, [None,None])
assert v1 == 'a\nb\nd\ne'
assert v2 == 'a\nc\nd'
assert cf == 2
assert names == ['HEAD', 'lala']

In [ ]:
#hide
tst = '\n'.join(['b', conflicts[1], 'c', f'{conflicts[2]} lala', 'd'])
v1,v2,cf,names = _split_cell(tst, 1, ['HEAD',None])
assert v1 == 'b\nd'
assert v2 == 'c\nd'
assert cf == 0
assert names == ['HEAD', 'lala']

In [ ]:
#hide
tst = '\n'.join(['c', f'{conflicts[2]} lala', 'd'])
v1,v2,cf,names = _split_cell(tst, 2, ['HEAD',None])
assert v1 == 'd'
assert v2 == 'c\nd'
assert cf == 0
assert names == ['HEAD', 'lala']

In [ ]:
#export
_re_conflict = re.compile(r'^<<<<<<<', re.MULTILINE)

In [ ]:
#hide
assert _re_conflict.search('a\nb\nc') is None
assert _re_conflict.search('a\n<<<<<<<\nc') is not None

In [ ]:
#export
def quasi_equal(v1, v2):
    if len(v1)==0 or len(v2)==0: return False 
    try:
        c1,c2 = json.loads(v1[:-1]),json.loads(v2[:-1])
        return c1['source']==c2['source']
    except Exception as e: return False

In [ ]:
#export
def analyze_cell(cell, cf, names, prev=None, added=False, fast=True, trust_us=True):
    "Analyze and solve conflicts in `cell`"
    if cf==0 and _re_conflict.search(cell) is None: return cell,cf,names,prev,added
    old_cf = cf
    v1,v2,cf,names = _split_cell(cell, cf, names)
    if fast and quasi_equal(v1,v2):
        if old_cf==0 and cf==0: return (v2 if trust_us else v1),cf,names,prev,added
        v1,v2 = (v2,v2) if trust_us else (v1,v1)
    res = []
    if old_cf == 0: 
        added=True
        res.append(get_md_cell(f'`{conflicts[0]} {names[0]}`'))
    res.append(v1)
    if cf ==0:
        res.append(get_md_cell(f'`{conflicts[1]}`'))
        if prev is not None: res += prev
        res.append(v2)
        res.append(get_md_cell(f'`{conflicts[2]} {names[1]}`'))
        prev = None
    else: prev = [v2] if prev is None else prev + [v2]
    return '\n'.join([r for r in res if len(r) > 0]),cf,names,prev,added

In [ ]:
#hide
tst = '\n'.join(['a', f'{conflicts[0]} HEAD', 'b', conflicts[1], 'c'])
c,cf,names,prev,added = analyze_cell(tst, 0, [None,None], None, False,fast=False)

In [ ]:
#export
def fix_conflicts(fname, fast=True, trust_us=True):
    "Fix broken notebook in `fname`"
    fname=Path(fname)
    shutil.copy(fname, fname.with_suffix('.ipynb.bak'))
    with open(fname, 'r') as f: raw_text = f.read()
    start,cells,end = extract_cells(raw_text)
    res = [start]
    cf,names,prev,added = 0,[None,None],None,False
    for cell in cells:
        c,cf,names,prev,added = analyze_cell(cell, cf, names, prev, added, fast=fast, trust_us=trust_us)
        res.append(c)
    if res[-1].endswith(','): res[-1] = res[-1][:-1]
    with open(f'{fname}', 'w') as f: f.write('\n'.join([r for r in res+[end] if len(r) > 0]))
    if fast and not added: print("Succesfully merged conflicts!")
    else: print("One or more conflict remains in the notebook, please inspect manually.")

## Export-

In [ ]:
#hide
notebook2script()

Converted 00_export.ipynb.
Converted 01_sync.ipynb.
Converted 02_showdoc.ipynb.
Converted 03_export2html.ipynb.
Converted 04_test.ipynb.
Converted 05_merge.ipynb.
Converted 06_cli.ipynb.
Converted 99_search.ipynb.
Converted index.ipynb.
